### 1. Import Libraries

In [1]:
import os
import sys
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

### 2. Import from mlcblab

In [2]:
from mlcvlab.models.nn1 import NN1
from mlcvlab.models.nn2 import NN2
from mlcvlab.nn.losses import l2
from mlcvlab.optim.sgd import SGD
from mlcvlab.optim.adam import Adam

### 3. Set Seed

In [3]:
np.random.seed(42)

### 4.Example code to load data and view image

In [4]:
def display_digit(x, index):
    some_digit = x[0]
    some_digit_image = some_digit.reshape((28, 28))

    plt.imshow(some_digit_image, cmap='binary')
    plt.axis('off')
    plt.show()

In [5]:
#x, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

In [6]:
#display_digit(x, 0)

In [7]:
#y


### 5. Helper functions

In [8]:
def load_dataset():
    '''Loads the whole dataset with true labels included.'''
    x, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
    return x,y

def prepare_data(x, y):
    '''Converts 10-ary labels in binary labels. If even then label is 1 otherwise 0.'''
    y = y.astype(int)
    y = y.reshape(len(y),1)
    y =  (y+1) % 2
    return x, y

def split_train_test(x,y):
    '''Partitioning the dataset into 10,000 test samles and the remaining 60,000 as training samples. 
    The shape  of the data will be M x N where M = 784 and N = 60000 for X and N = 10000 for y.'''
    #print(np.shape(x))
    #print(np.shape(y))
    
    X_train, X_test = x[:60000], x[60000:]
    y_train, y_test = y[:60000], y[60000:]

    #print("The size of X_train dataset in SPLIT is: ", np.shape(X_train))
    #print("The size of y_train dataset in SPLIT is: ", np.shape(y_train))
    #print("The size of X_test dataset in SPLIT is: ", np.shape(X_test))
    #print("The size of y_test dataset in SPLIT is: ", np.shape(y_test))

    return X_train, X_test, y_train, y_test


def initialize_model(X_train, X_test, y_train, y_test):
    '''Setting up the size of the weights/layer vectors.'''
    W = np.ones((np.shape(X_train)[0],1)) 
    one_layer_nn  = NN1()
    one_layer_nn.W = W
    one_layer_nn.layers[0].W = W

    return one_layer_nn

def train_model(model, X_train, y_train):
    '''Training the model using SGD or Adam optimizer.'''
    final_W  = None

    final_W = SGD(model, X_train, y_train, lr=0.1)
    #final_W = Adam(model, X_train, y_train)

    return final_W

def test_model(model, X_test, y_test, final_W):
    '''Tests the accuracy of the neural network.'''
    accuracy = None
    model.W = final_W
    
    #TODO: Call model.nn1 to test model.
    # get the predictions of the algorithm using testing x as the input

    # get the number of test instances
    T = np.shape(X_test)[0]

    A_total = np.zeros(shape=(np.shape(model.W)))

    for i in range(0,T):

        y_hat = model.nn1(X_test[i])

        A = abs(y_test - y_hat)

        # check if the value is greater than 0 and set it 1 if so.
        for x in range(len(A)):
                if A[x] > 0: 
                    A[x] = 1
        np.concatenate((A_total,A))
    
    # calculate the accuracy 
    accuracy = 1/T * np.sum(A_total)

    return accuracy

### 6. Run the program

In [9]:

#load data
x, y = load_dataset()

#prepare data
x, y = prepare_data(x,y)

# split data set
X_train, X_test, y_train, y_test = split_train_test(x,y)

#print("Shape of train_y before initialization: ",np.shape(y_train))


#initialize model
model = initialize_model(X_train, X_test, y_train, y_test)

#training model
final_W = train_model(model, X_train, y_train)
print(f"Completed training model - final W : {final_W}")

#testing model
accuracy = test_model(model, X_test, y_test, final_W)
print(f"Completed testing model - Accuracy : {accuracy}")    